In [22]:
# import csv
# import os
import numpy as np
import pandas as pd
# import gc
from datetime import timedelta, date
import uuid
import traceback
import matplotlib

# import rsa_data_summary as rd
# import rsa_data_wim as wim
# import rsa_headers as rh
import config
import queries as q
import tools

pd.options.mode.chained_assignment = None
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

import warnings
warnings.filterwarnings("ignore")

In [2]:
header_id = 'bba9b8bf-9db6-4970-95d3-80f72393af99'
select_qry = f"select * from trafc.electronic_count_data_partitioned_2022 where header_id = '{header_id}';"
SELECT_TYPE10_QRY = f"""SELECT * FROM trafc.electronic_count_data_type_10 t10
            left join traf_lu.vehicle_classes_scheme08 c on c.id = t10.vehicle_class_code_primary_scheme
            where t10.header_id = '{header_id}'
            """
AXLE_SPACING_SELECT_QRY = f"""SELECT 
            t10.id,
            t10.header_id, 
            t10.start_datetime,
            t10.edit_code,
            t10.vehicle_class_code_primary_scheme, 
            t10.vehicle_class_code_secondary_scheme,
            t10.direction,
            t10.axle_count,
            axs.axle_spacing_number,
            axs.axle_spacing_cm,
            wm.wheel_mass_number,
            wm.wheel_mass,
            vm.kg as vehicle_mass_limit_kg,
            sum(wm.wheel_mass) over(partition by t10.id) as gross_mass
            FROM trafc.electronic_count_data_type_10 t10
            left join trafc.traffic_e_type10_wheel_mass wm ON wm.type10_id = t10.data_id
            left join trafc.traffic_e_type10_axle_spacing axs ON axs.type10_id = t10.data_id and axs.axle_spacing_number = wm.wheel_mass_number
            Left join traf_lu.gross_vehicle_mass_limits vm on vm.number_of_axles = t10.axle_count
            where t10.header_id = '{header_id}' and wm.wheel_mass_number is not null
            """
WHEEL_MASS_SELECT_QRY = f"""SELECT 
            t10.id,
            t10.header_id, 
            t10.start_datetime,
            t10.edit_code,
            t10.vehicle_class_code_primary_scheme, 
            t10.vehicle_class_code_secondary_scheme,
            t10.direction,
            t10.axle_count,
            axs.axle_spacing_number,
            axs.axle_spacing_cm,
            wm.wheel_mass_number,
            wm.wheel_mass,
            vm.kg as vehicle_mass_limit_kg,
            sum(wm.wheel_mass) over(partition by t10.id) as gross_mass
            FROM trafc.electronic_count_data_type_10 t10
            inner join trafc.traffic_e_type10_wheel_mass wm ON wm.type10_id = t10.data_id
            Left join traf_lu.gross_vehicle_mass_limits vm on vm.number_of_axles = t10.axle_count
            Left join trafc.traffic_e_type10_axle_spacing axs ON axs.type10_id = t10.data_id
            where t10.header_id = '{header_id}'
"""

In [3]:
# df = pd.read_sql_query(select_qry, config.ENGINE)
df_AX = pd.read_sql_query(AXLE_SPACING_SELECT_QRY, config.ENGINE)
# df_WM = pd.read_sql_query(WHEEL_MASS_SELECT_QRY, config.ENGINE)

In [ ]:
df_AX.head()

In [60]:
mask = df_AX['id'].isin(df_AX.loc[(df_AX['axle_spacing_number'] == 1) & (df_AX['axle_spacing_cm'] <= 220), 'id'].tolist())
test_df = df_AX.loc[mask].copy()
test_df2 = test_df.loc[test_df.wheel_mass_number <= 2].groupby('id').sum()
test_df2.loc[test_df2.wheel_mass > 15000].count()

In [88]:
test_df = df_AX.loc[(df_AX['axle_spacing_number'] > 1) & (df_AX['axle_spacing_cm'] <= 220)].copy()

In [96]:
test_df = test_df.sort_values(by=['id','wheel_mass_number'])

In [97]:
test_df['check'] = test_df[['id','axle_spacing_number']].diff()['id']

In [ ]:
df_AX.loc[(df_AX['axle_spacing_number'] == 1) & (df_AX['axle_spacing_cm'] > 220) & (df_AX['vehicle_class_code_primary_scheme'].isin([4,5]))].sort_values('id')

In [ ]:
test_df.head(20)

In [ ]:
df_AX.loc[(df_AX['axle_spacing_number'] == 2)].groupby(df_AX['vehicle_class_code_primary_scheme'])[['axle_spacing_cm']].mean()

In [ ]:
df_AX.loc[(df_AX['axle_spacing_number'] != 1)].groupby(df_AX['id'])['axle_spacing_cm'].sum()*2.1+18000

In [ ]:
df_AX.loc[(df_AX['vehicle_class_code_primary_scheme'] == 7)].plot.scatter(x='axle_spacing_cm', y='axle_spacing_number')

In [4]:
nr_axle_groups = df_AX.loc[df_AX.axle_spacing_cm <= 220].groupby(df_AX.id)['id'].count()
nr_axle_groups.name = 'nr_axle_groups'
try:
    df_AX = df_AX.drop(columns=['nr_axle_groups'])
except:
    pass
df_AX = df_AX.join(nr_axle_groups, on='id')

In [ ]:
df_AX.head()

In [53]:
df_AX.loc[(df_AX['vehicle_class_code_primary_scheme'] == 4) & (df_AX['direction'] == 'P')].groupby('id').count().sum()[0]

816